In [ ]:
import os
import pandas as pd 

HOME = os.environ["HOME"]
CARDIAC_MOTION_GWAS_REPO = f"{HOME}/01_repos/CardiacMotionGWAS"
CARDIAC_MOTION_REPO = f"{HOME}/01_repos/CardiacMotionRL"
MLRUNS_DIR = f"{CARDIAC_COMA_REPO}/mlruns"
MSE_THRESHOLD = 0.8

runs_df = pd.read_csv(f"{CARDIAC_GWAS_REPO}/results/runs.csv")
runs_df = runs_df[runs_df["metrics.val_recon_loss"] < MSE_THRESHOLD]
runs_df = runs_df.set_index(["experiment_id", "run_id"], drop=False)

z_df_merged = []

for index, row in runs_df.iterrows():

    exp_id, run_id = row.experiment_id, row.run_id
    latent_vector_file = f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/latent_vector.csv"
    z_df = pd.read_csv(latent_vector_file)
    z_df = z_df.set_index("ID")
    z_df.columns = [ f"{z}_{run_id}" for z in z_df.columns ]
    z_df = z_df.sort_index()
    z_df.index = z_df.index.astype(int)
    z_df_merged.append(z_df)

z_df_merged = pd.concat(z_df_merged, axis=1)

z_df_merged.to_csv(
	f"{CARDIAC_MOTION_GWAS_REPO}/results/gwas_loci_summary_across_runs.csv", 
	index=True, index_label="ID", float_format='%.4f'
)